# Multiple model training, Pipeline and column Transformer

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')


In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
# what is Ml pipeline
# 1. ingest(read) the data
# 2. EDA
# 3. seperation of x and y
# 4. train test split
# 5. missing value treatment, outlier treatment, scaling
# 6. model training & evulation

In [4]:
# we want to solve the problem statement >> predict the what is time? if any one is visiting-- is
# it lunch or dinner? time is the target variable

df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [5]:
# EDA >> subjective

# encoding, missing value treatment, scalling >> outomate

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [7]:
df.isnull().sum()
# still we will see the null value imputation as well

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [8]:
# since time is nomial variable, then we will use label encoder

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,0,3
240,27.18,2.00,Female,Yes,Sat,0,2
241,22.67,2.00,Male,Yes,Sat,0,2
242,17.82,1.75,Male,No,Sat,0,2


In [9]:
df['time'].unique()  # dinner >> 1, lunch - 0

array([0, 1])

In [10]:
x = df.drop('time', axis = 1)
y = df['time']

In [11]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [12]:
x

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [13]:
# now x and y are ready then we doing train, test and split

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 1)

In [14]:
x_train.shape, x_test.shape

((195, 6), (49, 6))

In [15]:
y_train.shape, y_test.shape

((195,), (49,))

In [16]:
x_train.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
154,19.77,2.00,Male,No,Sun,4
167,31.71,4.50,Male,No,Sun,4
110,14.00,3.00,Male,No,Sat,2
225,16.27,2.50,Female,Yes,Fri,2


In [17]:
y_train.head()

0      0
154    0
167    0
110    0
225    1
Name: time, dtype: int64

In [18]:
x_test.head()

,total_bill,tip,sex,smoker,day,size
67,3.07,1.00,Female,Yes,Sat,1
243,18.78,3.00,Female,No,Thur,2
206,26.59,3.41,Male,Yes,Sat,3
122,14.26,2.50,Male,No,Thur,2
89,21.16,3.00,Male,No,Thur,2


In [19]:
# Handling the missing value  >> data transformer 
# data encoding  >> data transformer 
# feature scaling  >> data transformer 


from sklearn.impute import SimpleImputer   # simple way to treating the missing value
from sklearn.preprocessing import OneHotEncoder  # for encoding >> for categorical column
from sklearn.preprocessing import StandardScaler  # for scaling >> for numericla column


from sklearn.pipeline import Pipeline  # A sequence of data transform
from sklearn.compose import ColumnTransformer  # Group all the pipeline steps for each of the columns

In [20]:
df.sample(1)

,total_bill,tip,sex,smoker,day,time,size
162,16.21,2.0,Female,No,Sun,0,3


In [21]:
cat_cols = ["sex", "smoker", "day"]
num_cols = ["total_bill", "tip", "size"]

In [22]:
# feature engineering automation using pipeline and column transformer

num_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy='median')),
                  ('scaling', StandardScaler())])

cat_pipeline = Pipeline(steps = [('imputation', SimpleImputer(strategy='most_frequent')),
                  ('encoding', OneHotEncoder ())])

In [23]:
# now we both the pipeline combine together num_pipeline & cat_pipeline

preprocessor = ColumnTransformer([("num_pipeline", num_pipeline, num_cols),
                  ("cat_pipeline", cat_pipeline, cat_cols)])
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaling',
                                                  StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [24]:
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [25]:
x_train

array([[-0.28611937, -1.47443803, -0.57766863, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.02695905, -0.71612531,  1.47042924, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.3716196 ,  1.19880579,  1.47042924, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.23206267,  0.43283335, -0.57766863, ...,  0.        ,
         0.        ,  1.        ],
       [-1.06543688, -1.29060464, -0.57766863, ...,  1.        ,
         0.        ,  0.        ],
       [-0.29287646,  0.1034652 ,  0.44638031, ...,  1.        ,
         0.        ,  0.        ]])

In [26]:
x_test

array([[-1.85376383, -1.48209775, -1.60171757,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.08453291,  0.04984713, -0.57766863,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.79501474,  0.36389583,  0.44638031,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.        ],
       [-0.59356688, -0.33313909, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.18349826,  0.04984713, -0.57766863,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-1.32783714, -1.14506988, -0.57766863,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         0.   

In [27]:
# now data is ready , Lets build the multiple models

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [28]:
models = {"support vector classifier" : SVC(),
         "DT Classifier" : DecisionTreeClassifier(),
         "log reg" : LogisticRegression(),
         "RF" : RandomForestClassifier()}
models

{'support vector classifier': SVC(),
 'DT Classifier': DecisionTreeClassifier(),
 'log reg': LogisticRegression(),
 'RF': RandomForestClassifier()}

In [29]:
for i in range(len(models)):
    print(list(models.values())[i])

SVC()
DecisionTreeClassifier()
LogisticRegression()
RandomForestClassifier()


In [30]:
from sklearn.metrics import accuracy_score

def model_train_eval(x_train, x_test, y_trian, y_test, models):
    evalution = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        model_score = accuracy_score(y_pred, y_test)
        evalution[list(models.keys())[i]] = model_score
    return evalution
    
        
    
    
    

In [31]:
model_train_eval(x_train, x_test, y_train, y_test, models)

{'support vector classifier': 0.9183673469387755,
 'DT Classifier': 0.9387755102040817,
 'log reg': 0.9183673469387755,
 'RF': 0.8979591836734694}

# Random Forest Classifier

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf

RandomForestClassifier()

In [33]:
x_train, x_test

(array([[-0.28611937, -1.47443803, -0.57766863, ...,  0.        ,
          1.        ,  0.        ],
        [ 0.02695905, -0.71612531,  1.47042924, ...,  0.        ,
          1.        ,  0.        ],
        [ 1.3716196 ,  1.19880579,  1.47042924, ...,  0.        ,
          1.        ,  0.        ],
        ...,
        [-0.23206267,  0.43283335, -0.57766863, ...,  0.        ,
          0.        ,  1.        ],
        [-1.06543688, -1.29060464, -0.57766863, ...,  1.        ,
          0.        ,  0.        ],
        [-0.29287646,  0.1034652 ,  0.44638031, ...,  1.        ,
          0.        ,  0.        ]]),
 array([[-1.85376383, -1.48209775, -1.60171757,  1.        ,  0.        ,
          0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
          0.        ],
        [-0.08453291,  0.04984713, -0.57766863,  1.        ,  0.        ,
          1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          1.        ],
        [ 0.79501474,  0.363895

In [38]:
# here we doing haperparameter tunning because here a lot of parameter

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

params = {'max_depth' : [1,2,3,4,5,10,None],
         'n_estimators' : [30, 40, 50, 100, 200, 300],
         'criterion' : ['gini', 'entropy']}
params

{'max_depth': [1, 2, 3, 4, 5, 10, None],
 'n_estimators': [30, 40, 50, 100, 200, 300],
 'criterion': ['gini', 'entropy']}

In [39]:
clf = RandomizedSearchCV(rf, param_distributions=params, cv=5, verbose=3, scoring = 'accuracy', n_iter=10)

In [40]:
clf

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 2, 3, 4, 5, 10, None],
                                        'n_estimators': [30, 40, 50, 100, 200,
                                                         300]},
                   scoring='accuracy', verbose=3)

In [41]:
clf.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=30;, score=0.923 total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=30;, score=0.949 total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=30;, score=1.000 total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=30;, score=1.000 total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=30;, score=0.974 total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=30;, score=0.923 total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=30;, score=0.974 total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=30;, score=1.000 total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=30;, score=1.000 total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=30;, score=1.000 total time=   0.0s

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [1, 2, 3, 4, 5, 10, None],
                                        'n_estimators': [30, 40, 50, 100, 200,
                                                         300]},
                   scoring='accuracy', verbose=3)

In [42]:
clf.best_score_

np.float64(0.9794871794871796)

In [43]:
final_model = clf.best_estimator_
final_model.predict(x_test)

array([0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0])